In [ ]:
from pyspark import SparkContext
# initialize a new Spark Context to use for the execution of the script
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import re

In [ ]:
# EXERCISE
# Given a list of numbers separate odd numbers from even numbers

In [ ]:
# EXERCISE 
# Create a file with the following list of class,member pairs:
# 1,2;1,2;1,3;1,4;1,5;2,1;2,4;2,4;2,7;2,9;3,1;3,2;3,3;3,6;3,1
# Read the file and write the code to obtain the list of elements for each class

text_rdd = sc.textFile("ex1.txt")
pairs_rdd = text_rdd.flatMap(lambda x: x.split(";")).map(lambda x: x.split(",")).reduceByKey(lambda x,y: x+","+y)

print (pairs_rdd.collect())

In [ ]:
# #EXERCISE
# Read the txt file with the log of songs an user listened 
# to last week and find their top three favourite songs,
# the least favourite song and plot the distribution of
# the number of listens 

log_of_songs_rdd = sc.parallelize(log_of_songs)
playCounts = log_of_songs_rdd\
                .map(lambda x: (x, 1))\
                .reduceByKey(lambda x, y : x+y)
topSongs = playCounts.takeOrdered(3, (lambda t: -t[1]))
topSongs = [t[0] for t in topSongs]
leastFav = playCounts.takeOrdered(1, (lambda t: t[1]))[0]
print(topSongs)
print(f"The user's last week's top three songs are {topSongs[0]}, {topSongs[1]} and {topSongs[2]}")
print(f"The user's last week's least favourite song is {leastFav}")

orderedCounts = playCounts.takeOrdered(len(playCounts.collect()), (lambda t: -t[1]))
plt.figure(figsize=(15,5))
plt.bar(
    x=[l for l, freq in orderedCounts],
    height=[freq for l, freq in orderedCounts]
)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# EXERCISE
# Generate 1000 random points (x,y) and 20 seed points(a,b). 
# Assign the points to the closer seed and than 
# count how many points are assigned to each seed.

from random import random
import numpy as np

def closer(elem,listS):
    min = 1000000
    min_i = -1
    for e in listS:
        elem=np.array(elem)
        seed=np.array(e[1])
        p=np.sum((elem-seed)**2)
        d=np.sqrt(p)
        if min>d: 
            min = d
            min_i = e[0]
    return min_i

points = []
for _ in range(1000):
    value_x = random()
    value_y = random()
    points.append((value_x,value_y))
    
seeds = []
for i in range(20):
    value_x = random()
    value_y = random()
    seeds.append((i,(value_x,value_y)))
    
points_rdd = sc.parallelize(points)
distances_rdd = points_rdd.map(lambda x: closer(x, seeds)).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).sortByKey()

print(distances_rdd.collect())

In [ ]:
# EXERCISE
# Compute the numer of words appearing in both commedies and tragedies and the number of words which are not 
# present in both the texts

text1 = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/comedies.txt")
text2 = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/tragedies.txt")
words1 = text1.flatMap(lambda x: x.split())
words2 = text2.flatMap(lambda x: x.split())
both = words1.intersection(words2)

print(both.count())
print(words1.count()+words2.count()-both.count())

In [ ]:
# EXERCISE
# Read the file commedies and count the number of rows containing the word "skill"

text = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/comm2.txt")
pair_rdd = text.map(lambda x: x.lower().split()).filter(lambda x: "skill" in x)
print (pair_rdd.count())

In [ ]:
# EXERCISE
# Read the file commedies and count the number of words having the same second letter.

text = sc.textFile("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa00/comm2.txt")
pair_rdd = text.flatMap(lambda x: x.split()).filter(lambda x: len(x)>2).map(lambda x: (x[1].upper(), 1)).reduceByKey(lambda x,y:x+y)
print (pair_rdd.collect())

In [ ]:
shakespeare_rdd = sc.textFile("hdfs://masterbig-1.itc.unipi.it:54310/masterbig_data/shakespeare.txt")

In [4]:
#EXERCISE
#Clean text
pattern = re.compile("[^a-z0-9 ]+")
shakespeare_clean_rdd = shakespeare_rdd.map(lambda line: pattern.sub(' ', line.lower()))

NameError: name 'txt' is not defined

In [ ]:
# EXERCISE 
# Compute the number of distinct words
shakespeare_clean.flatMap(lambda line: line.split()).distinct().count()

In [ ]:
# EXERCISE
# Remove punctuation. Often punctuation is just noise, and it is here. Do a Map and/or Filter (some punctuation is
# attached to words, and some is not) to eliminate all punctuation from our Shakespeare data. Note that if you are
# familiar with regular expressions, Python has a ready method to use those.


In [ ]:
#EXERCISE
#Remove stop words
stop_words = ['a','and','the','is']
filtered_txt = shakespeare_rdd.flatMap(lambda x: x.split()).filter(lambda x: x not in stop_words)
filtered_txt.first()

In [ ]:
# EXERCISE
# Stemming. Recognizing that various different words share the same root ("run", "running") is important, but not so
# easy to do simply. Once again, Spark brings powerful libraries into the mix to help. A popular one is the Natural
# Language Tool Kit. You should look at the docs, but you can give it a quick test quite easily:


In [ ]:
## EXERCISE
# Compute the average word length of shakespeare.txt.
lengths_rdd = shakespeare_clean_rdd.flatMap(lambda line: [len(w) for w in line.split()])
print("The average word length is {:.2f}".format(lengths_rdd.sum() / lengths_rdd.count()))

In [3]:
# EXERCISE
# Compute the average wordlenght for each letter of the alphabet
avg_len_by_letter = shakespeare_clean\
    .flatMap(lambda line: [(w.lower()[0], len(w)) for w in line.split()])\
    .groupByKey()\
    .map(lambda x : (x[0], sum(list(x[1]))/len(list(x[1]))))\\
    .collect()
print(dict(avg_len_by_letter))
plt.figure(figsize=(15,5))
plt.bar(
    x=[letter for letter, avg in avg_len_by_letter],
    height=[avg for letter, avg in avg_len_by_letter]
)
plt.show()

NameError: name 'shakespeare_clean_rdd' is not defined

In [ ]:
# EXERCISE
# Compute the word length distribution of shakespeare.txt.
length_to_count = shakespeare_clean_rdd\
    .flatMap(lambda line: [(len(w), 1) for w in line.split()])\
    .reduceByKey(lambda l, r: l+r)\
    .collect()
print(dict(length_to_count))
plt.figure(figsize=(15,5))
plt.bar(
    x=[length for length, count in length_to_count],
    height=[count for length, count in length_to_count]
)
plt.show()

In [ ]:
# EXERCISE
#Find the most frequent words of shakespeare.txt.
shakespeare_clean_rdd\
    .flatMap(lambda line: [(w, 1) for w in line.split()])\
    .reduceByKey(lambda x, y: x+y)\
    .takeOrdered(10, (lambda pair: -pair[1]))

In [ ]:
# EXERCISE
#Compute term frequency (TF)
shakespeare_clean_rdd\
    .flatMap(lambda line: [(w, 1) for w in line.split()])\
    .reduceByKey(lambda x, y: x+y)\
    .takeOrdered(10, (lambda pair: -pair[1]))

In [ ]:
# Exercise
# Count the number of words of shakespeare.txt starting with each letter of the alphabet.
letters_to_count = shakespeare_clean_rdd\
    .flatMap(lambda line: [(w[0], 1) for w in line.split()])\
    .reduceByKey(lambda x,y: x+y)\
    .sortByKey()\
    .collect()
print(dict(letters_to_count))
plt.figure(figsize=(15,5))
plt.bar(
    x=[l for l, freq in letters_to_count],
    height=[freq for l, freq in letters_to_count]
)
plt.show()

In [ ]:
# EXERCISE
# Find the words pair most co-occurring next to each other of shakespeare.txt
def line_to_bigram(line):
    words = line.split()
    return [(words[i-1], words[i]) for i in range(1, len(words))]

shakespeare_clean_rdd\
    .flatMap(line_to_bigram)\
    .map(lambda bigram: (bigram, 1))\
    .reduceByKey(lambda x,y: x+y)\
    .takeOrdered(10, (lambda pair: -pair[1]))

In [ ]:
# EXERCISE
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))

#Compute term frequency (TF)
tot_word = shakespeare_clean.flatMap(lambda line: line.split()).count()
print(tot_word)
shakespeare_clean\
    .flatMap(lambda line: line.split())\
    .filter(lambda w: w not in stop_words)\
    .map(lambda w: (w, 1))\
    .reduceByKey(lambda x, y: x+y)\
    .mapValues(lambda x: x/tot_word)\
    .takeOrdered(20, (lambda pair: -pair[1]))

# Compute IDF????


In [ ]:
# EXERCISE
text_rdd = sc.textFile("/content/drive/MyDrive/input.txt")

# Filter the lines containing the word "google"
google = text_rdd.filter(lambda l: "google" in l)
google.collect()

# Output: the list of distinct IP addresses associated with the connections to a googlepage (i.e., connections to URLs containing the term “www.google.com”)
google = text_rdd.filter(lambda l: "google" in l).map(lambda line: line.split(" --")[0]).distinct()

# Output: report the maximum value of PM10
def split_and_get_temp(string):
    l = string.split(',')
    temp = l[2]
    return float(temp)
max_temp = text_rdd.map(split_and_get_temp).reduce(lambda t1,t2:t1 if t1>t2 else t2)
max_temp

# Output: the line(s) associated with the maximum value of PM10
def split_and_get_temp(string):
    l = string.split(',')
    temp = l[2]
    return float(temp)

text_rdd = sc.textFile("/content/drive/MyDrive/input.txt")
max_temp = text_rdd.map(split_and_get_temp).top(1)[0]
max_temp_vals = text_rdd.filter(lambda x : split_and_get_temp(x)==max_temp)
max_temp_vals.collect()

# Output: the date(s) associated with the maximum value of PM10
max_temp_vals = text_rdd.filter(lambda x : split_and_get_temp(x)==max_temp).map(lambda s: s.split(',')[1])
max_temp_vals.collect()

# Output: the maximum value of PM10 for each sensor
print(text_rdd.collect())
max_temp_by_sensor = text_rdd.map(lambda x: (x.split(',')[0], x.split(',')[2])).reduceByKey(lambda x, y: max(x, y))
max_temp_by_sensor = max_temp_by_sensor.collect()

In [ ]:
input_rdd = sc.parallelize([(1,2), (1,2), (1,3), (2,2), (2,4), (2,6)])
#How can you obtain the output?
# Sample output
# 1,2,2,3
# 2,2,4,6

input_rdd = input_rdd.groupByKey().mapValues(list).sortByKey().map(lambda t: str(t[0])+','+(','.join([str(v) for v in t[1]])))
output = '\n'.join(input_rdd.collect())
print(output)

In [5]:
# EXERCISE
# Create an RDD with integers from 1-50. 
# Apply a transformation to multiply every number by 2, 
# resulting in an RDD that contains the first 50 even numbers.

In [ ]:
# EXERCISE
# Word count problem
# input : 2 textual files
# output : number of occurrences of each word
# appearing in at least one file of the collection (i.e.,
# files of the input directory) and number of occurences of each word 
# appearing in both files